In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime
import requests

#function for dividing data for offers

zdroj = ["data analyst.html", "data engineer.html", "data scientist.html"]

def rozdeleni_zaznamu(zdroj):
    seznam_pozic = []
    
    try:
        for html in zdroj:
            nazev_pozice = html.split(".")[0]
            with open(html, "r") as file:
                obsah = file.read()
                data = bs(obsah, "html.parser")
                zaznamy = data.find_all("a", class_="posting-list-item")  
    
            for zaznam in zaznamy:
                seznam_pozic.append((nazev_pozice, zaznam))
        
        print("Seznam pozic byl vytvořen.")
        print("Bylo vytvořeno", len(seznam_pozic), "pozic.")
        
    except Exception:
        print("Něco se pokazilo. Zkontrolujte zdrojový kód.")
    return seznam_pozic



původní hledání (viz níže) nefunguje, protože je tam několik sekcí container, tudíž je potřeba skutečně jít už na ty položky - najde to jen položky v tom daném container, na který se to odkazuje a find_all a pak zase find_all nefungovalo. Možná by šlo find_all a pak pro každý záznam opět find_all a spojit to do seznamu, ale to je delší kód. 

data.find("div", class_= "list-container").find_all("a")

In [ ]:
#making a dictionary with information about the offer, correction of typos/translations etc.

import re

def slovnik_vytvoreni(zdroj):
    souhrn = []
    vysledek = rozdeleni_zaznamu(zdroj)
  
    for kategorie, nabidka in vysledek:
        jmeno = nabidka.find("h3", class_="posting-title__position text-truncate").text           #proč v předchozím vyhledávání (předchozí funkce) nemusel být celý název třídy?  - tam kde je find_all se dá jen ta společná část názvu, u find se musí dát celý název
        
        lokalita = nabidka.find("span", class_="tw-text-ellipsis tw-inline-block tw-overflow-hidden tw-whitespace-nowrap tw-max-w-[100px] md:tw-max-w-[200px] tw-text-right").text
        if "Zdalnie" in lokalita:
            lokalita = "Remote"
        dalsi_lokality = (nabidka
                          .find_all("div", class_ = "tw-flex tw-items-center ng-star-inserted"))
        print(dalsi_lokality)
        lokalita_2 = dalsi_lokality[0].text
        nr_dalsich_lokalit = lokalita_2.split("+")[-1] if len(lokalita_2.split("+")) > 1 else "0"  
    


        firma = nabidka.find("h4", class_="tw-text-gray-60 company-name tw-w-[50%] lg:tw-w-auto tw-mb-0 !tw-text-xs !lg:tw-text-sm tw-font-semibold lg:tw-font-normal").text

        mzda = nabidka.find("span", class_="salary")
        ocistena_mzda = re.sub(r"\s+", "", mzda.text)
        print(ocistena_mzda)
        rozsah = re.findall(r"[0-9]+", ocistena_mzda)
        print(rozsah)
        try:
            minimum = rozsah[0]
            maximum = rozsah[1] if len(rozsah) > 1 else rozsah[0]
            mena = re.findall(r"[A-Z]+", ocistena_mzda)[0]    #šlo by to i jako mzda.text.split()[-1] (řešení dle Moniky)
        except:
            minimum = None
            maximum = None
            mena = None
        technologie = nabidka.find("div", class_="tiles-container lg:tw-w-[50%] lg:tw-pr-3 tw-flex-[0_0_50%]").text

        info = {"nazev pozice" : jmeno,
                "kategorie" : kategorie, 
                "nazev firmy" : firma, 
                "technologie" : technologie, 
                "lokalita" : lokalita,
                "počet dalsich lokalit" : nr_dalsich_lokalit, 
                "mzda" : ocistena_mzda,
                "dolni hranice" : minimum,
                "horni hranice" : maximum,
                "mena" : mena} 
            
        souhrn.append(info)

    return souhrn

seznam = slovnik_vytvoreni(zdroj)
seznam   






In [ ]:
#making of CSV file with job offers and data for them for specific position

import os

df = pd.DataFrame(seznam)
df   

try:
    current_date = datetime.datetime.now()
    current_date.strftime("%Y_%m_%d")
    nazev_souboru = f"job_offers_{current_date.date()}"
    df.to_csv(f"{nazev_souboru}.csv", index=False)
    print("Dokument byl vytvoren.")
except:
    print("Dokument nebyl vytvořen, zkontroluj kód.")
    raise


In [ ]:
data = pd.read_csv("job_offers_2024-06-27.csv",
                   encoding = "utf-8",
                   sep = ",",
                   decimal = "."
                   )

data